<center>
<img src="img/ml_theme.png">
# Вводный курс по нейронным сетям
<center>
**Автор материала: программист-исследователь Mail.Ru Group  Юрий Кашницкий**

# <center>Часть 4. Эффективное обучение нейронных сетей
<center>Данный материал создан на базе [лекций](https://habrahabr.ru/company/mailru/blog/344982/) "Техносферы"
</center> 

**План:**
1. Инициализация весов
 - Xavier
 - He
2. "Умная" регуляризация
 - Dropout
 - Batch Normalization
3. Функции активации
4. Продвинутые методы оптимизации
 - SGD + Momentum
 - Adagrad
 - RMSProp
 - Adam

# 1. Инициализация весов
## Инициализация Xavier (Glorot)

> *"Understanding the difficulty of training deep feedforward neural networks"* [X. Glorot, Y. Bengio](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf) (University of Montreal, 2010)

Для нечетной функции активации с единичной производной в нуле, например, `tanh`.

<img src='img/tanh_and_sigmoid.png' width=80%>

Дисперсия активаций и градиентов (вывод у доски):

$$\large z^{i+1} = f(z^iW^i)$$
$$\large \mathbb{D}[z^i] = \mathbb{D}[x]\prod_{k=0}^{i-1}n_k\mathbb{D}[W^k]$$
$$\large \mathbb{D}[\frac{\partial L}{\partial s^i}] = \mathbb{D}[\frac{\partial L}{\partial s^d}] \prod_{k=i}^{d}n_{k+1}\mathbb{D}[W^k]$$
Здесь $n_i$ – размерность $i$-го слоя.

Хорошей инициализацией будет такая, что градиенты не взрываются и не затухают, то есть:

$$\forall i, j \  \mathbb{D}[z^i] = \mathbb{D}[z^j],\  \mathbb{D}[\frac{\partial L}{\partial s^i}] = \mathbb{D}[\frac{\partial L}{\partial s^j}]  $$

Получается несовместная система условий 

$$\forall i \ n_i \mathbb{D}[W^i] = 1, \ n_{i+1} \mathbb{D}[W^i] = 1  $$ 

с компромиссом (средним геометрическим) $\mathbb{D}[W^i] = \frac{2}{n_i + n_{i+1}}$, откуда следует, что веса можно брать, из равномерного распределения: $\large W^i \sim U[-\frac{\sqrt{6}}{\sqrt{n_i + n_{i+1}}}, \frac{\sqrt{6}}{\sqrt{n_i + n_{i+1}}}],$ поскольку дисперсия равномерного распределения $\mathbb{D}[U(a,b)] = \frac{1}{12}{(b-a)}^2$.

## Инициализация He

> *"Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification"*, [K. He, X. Zhang et.al](https://arxiv.org/abs/1502.01852) (Microsoft Research, 2015)


Инициализация He – специально для активации ReLU.

Можно провести аналогичные рассуждения и получить:

$$\mathbb{D}[z^i] = \mathbb{D}[x]\prod_{k=0}^{i-1}\frac{1}{2}n_k\mathbb{D}[W^k] \Rightarrow \mathbb{D}[W^k] = \frac{2}{n_k}$$
$$\mathbb{D}[\frac{\partial L}{\partial s^i}]= \mathbb{D}[\frac{\partial L}{\partial s^d}]\prod_{k=i}^{d}\frac{1}{2}n_{k+1}\mathbb{D}[W^k] \Rightarrow \mathbb{D}[W^k] = \frac{2}{n_{k+1}}$$

Можно взять только первое уравнение, и тогда 
$$\mathbb{D}[\frac{\partial L}{\partial s^i}] = \frac{n_2}{n_d}\mathbb{D}[\frac{\partial L}{\partial s^d}],$$ 
$$W^i \sim N(0, \frac{2}{n_i})\hspace{1cm} \text{или} \hspace{1cm} W^i \sim N(0, \frac{2}{n_{i+1}})$$
На практике для сверточных сетей (в которых почти всегда используется ReLU) отношение $\frac{n_2}{n_d}$ небольшое, и поэтому дисперсия градиентов убывает не быстро.

Далее картинки из оригинальной статьи [K. He, et.al.](https://arxiv.org/abs/1502.01852) – сравнение инициализаций He и Xavier для 22-слойной и 30-слойной сетей. В обоих случаях ReLU-активация. В случае 30 слоев получилось даже, что сеть сошлась при инициализации He, но не сошлась с инициализацией Xavier.
<img src='img/he_versus_xavier22.png' width=70%>

<img src='img/he_versus_xavier30.png'>

# 2. "Умная" регуляризация

## Dropout

> *"Dropout: A Simple Way to Prevent Neural Networks from Overfitting"*, [N. Srivastava, G. Hinton, A. Krizhevsky, I. Sutskever, R. Salakhutdinov](http://jmlr.org/papers/volume15/srivastava14a.old/srivastava14a.pdf) (University of Toronto, 2014)

Все вроде бы очень просто: с некоторой вероятностью $p$ зануляем активацию каждого нейрона в сети. 

<img src="./img/dropout1.png" >

Мотивация и интерпретации:
 - В мозге человека "включены" далеко не все связи аксонов с дендритами, иначе мозг тратил бы слишком много энергии, и мы были бы ленивцами
 - Борьба с соадаптацией – нейроны больше не могут рассчитывать на наличие соседей
 - Биология: не все гены родителей будут присутствовать у потомков
 - Усреднение астрономически большого числа моделей ($2^n$)
<img src="./img/dropout2.png" >

<img src="./img/dropout3.png" >

```python
class torch.nn.Dropout
class torch.nn.Dropout2d(
    p=0.5,         # % связей, которые нужно занулить
    )
```

Одна из вариаций: `DropConnect`. Тут с некоторой вероятностью зануляем каждую связь в сети.
<img src="./img/dropconnect.png">

## Batch Normalization

> *"Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift"*, [S.Ioffe, C.Szegedy](https://arxiv.org/pdf/1502.03167.pdf), (Google Inc., 2015)

<img src="./img/batchnorm1.png" >

Мы обсуждали проблемы градиентного спуска, когда данные не нормализованы. Можно показать, что это происходит не только с входными данными, но и с выходами слоёв, даже если входные данные нормализованы (см [Andrej Karpathy cs231n](https://youtu.be/gYpoJMlgyXA?list=PLkt2uSq6rBVctENoVBg1TpCC7OQi31AlC&t=2221)).

Решение проблемы – нормализация данных внутри сети, Batch Normalization 

<img src="./img/batchnorm2.png" width=400>

$$
y = \frac{x - \mathbb{E}[x]}{ \sqrt{\mathbb{D}[x] + \epsilon}} * \gamma + \beta
$$

На практике $\mathbb{E}[x]$ и $\mathbb{D}[x]$ считаются по всей выборке как скользящие средние.

Backprop через слой BatchNorm-слой (разбор у доски)

<img src="./img/batchnorm_backprop.png" width=600>

Результаты экспериментов с ImageNet и сетью Inception ([Ioffe, Szegedy 2015](https://arxiv.org/pdf/1502.03167.pdf)):
 - BN-Baseline – Inception с BN перед каждой нелинейностью
 - BN-x5 – Inception с BN и темпом обучения, увеличенным в 5 раз (при таком темпе обучения оригинальный Inception, без BN, просто не учится)
 - BN-x30 – то же самое, но с темпом обучения, увеличенным в 30 раз
 - BN-x5-Sigmoid – то же, что BN-x5б но с $\sigma$-нелинейностью, при которой Inception без BN не выучивал толком ничего

<img src="./img/batchnorm_inception.png" width=400>


```python
class torch.nn.BatchNorm2d(
    num_features,   # Число фильтров, которое следует ожидать на входе
    eps=1e-05,      # Epsilon из знаменателя
    momentum=0.1,   # Момент, с которым идет накопление среднего и дисперсии
    affine=True)    # учить gamma и beta
```

Детали:
- Надо убрать смещения
- Стоит использовать большие значения learning rate в начале обучения, а затем уменьшать
- При BatchNorm другие регуляризаторы (dropout и L2) стоит уменьшить
- Надо перемешивать обучающую выборку (чтоб не переобучаться при обучении $\gamma$ и $\beta$)

# 3. Функции активации

<img src='img/all_activation_functions.png'>
```python
class torch.nn.Sigmoid
```
Чем плох сигмоид?
<img src="./img/sigmoid.png" >
<img src="./img/sigmoid_deriv.png" >

```python
class torch.nn.ReLU(inplace=False)
```
<img src="./img/relu.png" >

Активация на выходе в задаче классификации – почти всегда `Softmax`.
```python
class torch.nn.Softmax
class torch.nn.Softmax2d
```
$$\large {Softmax}_j(x) = \frac{exp(x_j)}{\sum_i exp(x_i)}$$

# 4. Продвинутые методы оптимизации
## SGD + Momentum
$$
\Large
\begin{align}
m &\leftarrow  \alpha m+ (1 - \alpha) \nabla_{\theta} L \\
\theta & \leftarrow \theta - \eta m
\end{align}
$$
<img src="img/sgd_momentum.gif" width=500>




## Adagrad
В некоторых задачах хотим подстраивать темп обучения для каждого веса по отдельности. Например, в текстах для редких слов использовать больщой темп обучения, а для частых – маленький. Эта идея реализована в методе Adagrad, где для каждого веса хранится сумма квадратов градиентов по нему, и корень из этой суммы участвует в знаментале в формуле пересчета весов
<img src='img/adagrad.png'>

## RMSProp (Root Mean Square Propagation)
Тут в целом то же самое, что в Adagrad, только считаем экспоненциальное среднее для квадратов градиентов. 
$$
\Large
\begin{align}
v &\leftarrow  \beta v +  (1-\beta)(\nabla_{\theta} L)^2 \\
\theta & \leftarrow \theta - \eta \frac{1}{\sqrt{v} + \epsilon} \nabla_{\theta} L
\end{align}
$$
  
<img src="img/optim_comparison.gif" width=500>

Выход из седловой точки
<img src='img/saddle_point.gif'>

## Adam (Adaptive Moment Estimation) 

Adam = SGD + Momentum + RMSProp. Почти всегда используется именно он.
Как и раньше, к градиентам и их квадратам применяется экспоненциальное сглаживание.
$$
\Large
\begin{align}
m &\leftarrow  \alpha m+ (1 - \alpha) \nabla_{\theta} L \\
v &\leftarrow  \beta v +  (1-\beta)(\nabla_{\theta} L)^2 
\end{align}
$$

Только теперь делается еще поправки для несмещенности: $\mathbb{E}[m_t] = \mathbb{E}[\nabla_{\theta} L_t] $ и $\mathbb{E}[v_t] = \mathbb{E}[{(\nabla_{\theta} L_t)}^2]$:

$$\hat{m_t} = \frac{m_t}{1 - \alpha^t} \hspace{1cm} \hat{v_t} = \frac{v_t}{1 - \beta^t}$$

Интуиция: вначале скорость высокая, в конце – снижается. 
Формула пересчета весов:
$$ \Large
\theta  \leftarrow \theta - \eta \frac{\hat{m}}{\sqrt{\hat{v}} + \epsilon} 
$$

Хорошие (на практике) значения: $\alpha$=0.9, $\beta$=0.9, $\epsilon = 10^{-8}$